<a href="https://colab.research.google.com/github/Lua-Nova/Modern-GAP-GNN/blob/main/ModernGAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
if torch.cuda.is_available():
  #NVIDIA GPU version
  %pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f f'https://data.pyg.org/whl/torch-1.12.0+{cutorch.version.cuda.replace('.','')}.html'
else:
  #CPU version
  %pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cpu.html
%pip install opacus

device = "cuda" if torch.cuda.is_available() else "cpu"



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cpu.html
     |████████████████████████████████| 286 kB 4.1 MB/s 
     |████████████████████████████████| 641 kB 9.1 MB/s 
     |████████████████████████████████| 311 kB 45.3 MB/s 
     |████████████████████████████████| 121 kB 8.5 MB/s 
     |████████████████████████████████| 467 kB 4.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=02b187fc7f1a045e1e6c089b1bc5606b823f1db0ae2f414d08901f02d6f6a927
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 4.6 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:49tcmalloc

In [ ]:
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.nn import Sequential, GCNConv
import opacus as op

## Encoder Module

In [ ]:
torch.manual_seed(11)
# create classes for layers that are used a lot to avoid repeating code

class MLP(nn.Module):
  # e.g. dimensions = [50,40,30,20]
    def __init__(self, dimensions):
        super().__init__()
        self.flatten = nn.Flatten()
        layers = []
        for i in range(len(dimensions)-1):
          layers.append(nn.Linear(dimensions[i], dimensions[i+1]))
          layers.append(nn.SELU(inplace=True))

        self.linear_relu_stack = nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## PMA

In [ ]:
class PMA(nn.Module):
    # A - adjacency matrix     TODO: this should not be given to the module itself, it should access it in training (or from the graph dataset)
    # num_hops - the number of hops covered by this GNN
    def __init__(self, num_hops, sigma):
        super().__init__()
        # TODO: Figure out if you should tranpose this
        # self.A_transpose = torch.transpose(A, 0,1)
        self.num_hops = num_hops
        self.sigma = sigma
        self.A = None
    
    def set_adj_mat(self, A):
        self.A = A

    def forward(self, x):
        if self.A is None:
          raise RuntimeError("Set adjacency matrix before running")
        out = [torch.nn.functional.normalize(x, dim=1)]
        for k in range(self.num_hops):
            aggr = torch.mm(self.A, out[-1])
            noised = aggr + torch.normal(torch.zeros(aggr.size()), std=self.sigma)
            normalized = torch.nn.functional.normalize(noised, dim=1)
            out.append(normalized)
        return torch.stack(out)
        # return torch.nn.functional.normalize(x, dim=1)

In [ ]:
class PMAT(nn.Module):
    def __init__(self, num_hops, transform_dimensions, attention_dimensions):
        self.num_hops = num_hops
        self.A = None
        self.transforms = nn.ModuleList()
        self.attentions = nn.ModuleList()
        for i in range(num_hops):
          self.transforms.append(nn.Linear(*transform_dimensions)) # Only 1 layer transformation
          self.attentions.append(MLP(attention_dimensions))

    def set_adj_mat(self, A):
        pass

    def forward(self, x):
        if self.A is None:
          raise RuntimeError("Set adjacency matrix before running")
        out = [torch.nn.functional.normalize(x, dim=1)]
        h = x
        for k in range(self.num_hops):

            # torch.sparse.sum(torch.index_select(A, 0, torch.tensor([0, 1, 2, 3])))

            neighbour_filter = neighbour_indices[node] # self.A[node, :]

            # e = torch.zeros(self.dimensions.size(dim=1))

            h = self.transforms[k](h)

            e = self.attentions[k](h[edge_index.T])

            

            # edge_indices[node]
            # node_indices_of_neighbours[node]

            # node_indices_of_neighbours[node] = edge_index[edge_indices[node]][:, 1]

            alpha[edge_indices[node]] = torch.Softmax(e[edge_indices[node]], dim=0)

            # alpha[edge_indices[nodes]] = torch.exp(e[edge_indices[nodes]]) / torch.sum(torch.exp(e[edge_indices[nodes]]))

            h_prime[nodes] = torch.sum(alpha[edge_indices[nodes]] * h[node_indices_of_neighbours[nodes]])





            aggr = torch.mm(self.A, out[-1])
            noised = aggr + torch.normal(torch.zeros(aggr.size()), std=self.sigma)
            normalized = torch.nn.functional.normalize(noised, dim=1)
            out.append(normalized)
        return torch.stack(out)


In [ ]:
# # TEMP CODE
# smoothing = 0.2
# A = torch.tensor([[1.,smoothing,smoothing],
#                   [smoothing,1.,smoothing],
#                   [smoothing,smoothing,1.]])
# x = torch.tensor([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]])
# pma = PMA(10, 1)
# tensor = pma(x, A)
# tensor = tensor.cpu().numpy()

# # plt.figure(figsize=(16,7))
# # plt.imshow(tensor)
# # plt.show()
#         # [encoder, pma, element_wise_mlp, combine, mlp]


In [ ]:
10*torch.ones((2, 3)) + torch.normal(torch.zeros((2, 3)), std=1)

tensor([[10.7376, 11.9459,  9.3005],
        [ 8.6977,  9.4867,  9.7304]])

## Classification Module
NOTE: 

MLP base: The first MLP in the cassification module. 

MLP head: The last MLP and takes the combined output of all MLP base.

In [ ]:
class Classification(nn.Module):
    # num_hops - the number of hops covered by this GNN
    # encoder_dimensions - the MLP dimensions of each base MLP
    # head_dimensions - the dimensions of the head MLP
    def __init__(self, num_hops, encoder_dimensions, head_dimensions):
        super().__init__()
        self.base_mlps = nn.ModuleList()
        for i in range(num_hops+1):
          self.base_mlps.append(MLP(encoder_dimensions))
        self.head_mlp = MLP(head_dimensions) # TODO: should this be softmax? I think we add a softmax for classification tasks. We can test if it works better
    
    def forward(self, cache):
        # forward through bases
        out = []
        for i in range(len(self.base_mlps)):
          encoding = self.base_mlps[i](cache[i,:,:])
          out.append(encoding) # add corresponding encoding
          # TEMP
          # out.append(cache[i, :, :])
        # combine (use concatenation)
        combined_x = torch.cat(out, dim=1)
        # forward through head
        return self.head_mlp(combined_x)

In [ ]:
class GAP(nn.Module):
  # encoder - pretrained encoder module
  # pma - PMA module
  # classification - classification module
  def __init__(self, encoder, pma, classification): # TODO: decide whether we should recieve the models as parameters
    super().__init__()
    self.encoder = encoder
    self.encoder.requires_grad=False
    self.pma = pma
    self.classification = classification

  def set_adj_mat(self, A):
    self.pma.set_adj_mat(A)

  def forward(self, x):
    # initial node encoding
    x_encoded = self.encoder(x)
    # aggregation module
    cache = self.pma(x_encoded) 
    # classification
    return self.classification(cache) 


##Hyperparameters

In [ ]:
node_level = True

# Edge level DP
epsilon, delta, alpha = 1000, 0.1, 1
# specify specific epsilon_1, epsilon_5 for node-level and then just do a subtraction, and calculate sigma from remaining epsilon
K_hop = 5
# sigma = 1 / np.max(np.roots([K_hop/2, np.sqrt(2*K_hop*np.log(1/delta)), -epsilon]))
sigma = 0
# Node level DP
if (node_level):
  pass
  # How do we calculate this?
data = "reddit"

print("sigma:", sigma)

sigma: 0


## Data

In [ ]:
from torch_geometric.data import Data

# this method partitions based on nodes (so edges between splits are not used)
def train_test_split(dataset, test_ratio):
    X, y, edge_index= dataset.x, dataset.y, dataset.edge_index
    shuffle_ordering = torch.randperm(X.size(dim=0))

    edge_mapping = torch.zeros(X.size(dim=0), dtype=torch.long)
    edge_mapping[shuffle_ordering] = torch.arange(X.size(dim=0))

    X = X[shuffle_ordering]
    y = y[shuffle_ordering]
    edge_index = edge_mapping[edge_index]

    mask = torch.zeros(X.size(dim=0), dtype=torch.bool)
    train_slice = int((1-test_ratio)*X.size(dim=0))
    mask[:train_slice] = True

    X_train = X[mask]
    X_test = X[~mask]

    y_train = y[mask]
    y_test = y[~mask]

    edge_index_train = edge_index[:, torch.logical_and(*mask[edge_index])]
    edge_index_test = edge_index[:, torch.logical_and(*~mask[edge_index])] - train_slice

    return Data(x=X_train, y=y_train, edge_index=edge_index_train), \
           Data(x=X_test, y=y_test, edge_index=edge_index_test)


# returns filtered edge index, first removes edges that have removed src or dst nodes, then shifts indices of remained src/dst nodes
def filter_edge_index(edge_index, filter):

    node_indices = torch.arange(filter.size(dim=0))[filter]
    edge_mapping = torch.zeros(filter.size(dim=0), dtype=torch.long)
    edge_mapping[node_indices] = torch.arange(node_indices.size(dim=0))


    edge_index = edge_index.to(torch.long)
    edge_filter = torch.logical_and(*filter[edge_index])
    return edge_mapping[edge_index[:, edge_filter]]

def prepare_dataset(dataset, threshold):
    X, y, edge_index = dataset.x, dataset.y, dataset.edge_index

    # remove labels with less examples than threshold
    index_map = torch.zeros(y.size())
    included_classes = y.unique(return_counts=True)[1] >= threshold
    filter = included_classes[y]
    # remap labels (i.e. if they were 0-8 and we remove 4 labels, new labels should be between 0 and 4)
    label_mapping = torch.zeros(included_classes.size(dim=0), dtype=torch.long)
    label_mapping[included_classes] = torch.arange(torch.count_nonzero(included_classes))

    y = label_mapping[y[filter]].to(torch.long)
    X = X[filter]

    # remove edges that had their nodes removed
    edge_index = filter_edge_index(edge_index, filter)

    return Data(x=X, y=y, edge_index=edge_index)

def get_adjacency_matrix(dataset):
    edge_index = dataset.edge_index

    # make sparse adjacency matrix, A
    values = torch.ones(edge_index.size(dim=1), dtype = torch.int)
    A = torch.sparse_coo_tensor(edge_index, values, (dataset.x.size(dim=0), dataset.x.size(dim=0)), dtype=torch.float)

    return A

In [ ]:
from torch_geometric.datasets import Amazon
from torch_geometric.loader import DataLoader

dataset = Amazon('.', name='Computers')[0]
# prepare dataset by removing classes that have less than 1000 examples
dataset = prepare_dataset(dataset, 1000)
# get num classes
num_classes = torch.unique(dataset.y).size(dim=0)

# train/test split
train_dataset, test_dataset = train_test_split(dataset, 0.2)

Processing...
Done!


In [ ]:
edge_index = torch.tensor([[0, 2, 0, 0, 2, 3, 1, 4, 1, 2, 4],
                           [1, 0, 3, 4, 1, 1, 4, 4, 1, 3, 3]], dtype=torch.long)
x = torch.tensor([[0], [1], [2], [3], [4]], dtype=torch.float)
y = torch.tensor([0, 1, 1, 2, 2], dtype=torch.long)

data = Data(x=x, y=y, edge_index=edge_index)

# filter_edge_index(data.edge_index, torch.tensor([False, True, True, False, True]))

# train_data, test_data = train_test_split(data, 0)

data = prepare_dataset(data, 1)
A = get_adjacency_matrix(data)

torch.sparse.sum(torch.index_select(A, 0, torch.tensor([0, 1, 2, 3])))

# .to_dense()
# print(data.y)
# print(data.x)

tensor(9.)

In [ ]:
X_train, y_train, X_test, y_test = train_dataset.x, train_dataset.y, test_dataset.x, test_dataset.y
A_train, A_test = get_adjacency_matrix(train_dataset), get_adjacency_matrix(test_dataset)

## Train/Test


In [ ]:
# train
def train(X, y, model, loss_fn, optimizer): 
    # make this into dataloader using backup
    model.train()
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# test
def test(X, y, split, model, loss_fn):
    size = X.size(dim=0)
    model.eval()
    test_loss, correct = 0, 0
    with torch.inference_mode():
        X, y = X, y
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct /= size
    print(f"{split.title()} Error: \n Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")

# graph train
def graph_train(X, y, A, model, loss_fn, optimizer): 
    # make this into dataloader using backup
    model.train()
    model.set_adj_mat(A)
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# graph test
def graph_test(X, y, A, split, model, loss_fn):
    size = X.size(dim=0)
    model.set_adj_mat(A)
    model.eval()
    test_loss, correct = 0, 0
    with torch.inference_mode():
        X, y = X, y
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct /= size
    print(f"{split.title()} Error: \n Accuracy: {(100*correct):>0.1f}%, Loss: {test_loss:>8f} \n")

##Sampling from K-hop neighbourhood

## Encoder

Encoder Design


In [ ]:
# encoder
dimensions = [767, 300, 60]
encoder_train = nn.Sequential(
    MLP(dimensions),
    nn.Linear(dimensions[-1], num_classes),
    nn.Softmax(dim=1)
)

Encoder Pretraining

In [ ]:
encoder_model = encoder_train.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(encoder_model.parameters(), lr=1e-3)

# if node_level:
#   optimizer = op.optimizers.optimizer.DPOptimizer(
#       # TODO: Fill out these parameters '?'
#       optimizer=optimizer,
#       noise_multiplier=?,
#       max_grad_norm=?
#   )

epochs = 100
for t in range(epochs):
    train(X_train, y_train, encoder_model, loss_fn, optimizer)
    if (t + 1) % 10 == 0:
      test(X_train, y_train, "TRAIN", encoder_model, loss_fn)
test(X_test, y_test, "TEST", encoder_model, loss_fn)
print("Done!")

encoder = encoder_model[0]

# for name, param in encoder_model.named_parameters():
#     if param.requires_grad:
#         print(name, param.data)

Train Error: 
 Accuracy: 91.4%, Loss: 0.835441 

Train Error: 
 Accuracy: 93.5%, Loss: 0.813172 

Train Error: 
 Accuracy: 94.8%, Loss: 0.801979 

Train Error: 
 Accuracy: 95.1%, Loss: 0.799237 

Train Error: 
 Accuracy: 95.3%, Loss: 0.795687 

Train Error: 
 Accuracy: 95.5%, Loss: 0.792990 

Train Error: 
 Accuracy: 95.7%, Loss: 0.790494 

Train Error: 
 Accuracy: 95.9%, Loss: 0.788637 

Train Error: 
 Accuracy: 96.0%, Loss: 0.787141 

Train Error: 
 Accuracy: 96.1%, Loss: 0.785771 

Test Error: 
 Accuracy: 91.4%, Loss: 0.830850 

Done!


## Full Model Training

TODO:
  - try other dataset
  - compare with their implementation
  - try train/test split across edges
  - batch normalization
  - add identity matrix to see performance

Train full model

In [ ]:
encoder.requires_grad=False
# model = nn.Sequential(encoder, 
#                       PMA(K_hop, sigma), 
#                       nn.Linear(60, num_classes), 
#                       nn.Softmax(dim=1))
# model = nn.Sequential(encoder,
#                       PMA(K_hop, sigma),
#                       Classification(K_hop, [60, 20], [(K_hop+1)*20, num_classes]))
model = GAP(encoder, 
            PMA(K_hop, sigma), 
            Classification(K_hop, [60, 20], [(K_hop+1)*20, num_classes]))
# model = nn.Sequential(encoder, 
#                       PMA(K_hop, sigma))
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()

# print(model(X))

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

epochs = 10
# model.set_adj_mat(A_train)
for t in range(epochs):
    graph_train(X_train, y_train, A_train, model, loss_fn, optimizer)
    if (t + 1) % 2 == 0:
      graph_test(X_train, y_train, A_train, "TRAIN", model, loss_fn)
# model.set_adj_mat(A_test)
graph_test(X_test, y_test, A_test, "TEST", model, loss_fn)
print("Done!")

Train Error: 
 Accuracy: 95.6%, Loss: 0.160033 

Train Error: 
 Accuracy: 95.6%, Loss: 0.159795 

Train Error: 
 Accuracy: 95.6%, Loss: 0.159560 

Train Error: 
 Accuracy: 95.6%, Loss: 0.159328 

Train Error: 
 Accuracy: 95.6%, Loss: 0.159099 

Test Error: 
 Accuracy: 91.4%, Loss: 0.256538 

Done!


In [ ]:
# Sigma calculated above in node-level and edge-level DP case
gap = GAP(encoder, PMA(A, K_hop, sigma), Classification(K_hop, [60, 30, 20], [(K_hop+1)*20, 60, 30, num_classes]))
gap_model = gap.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(gap_model.parameters(), lr=1e-1)

# if node_level:
#   optimizer = op.optimizers.optimizer.DPOptimizer(
#       # TODO: Fill out these parameters '?'
#       optimizer=optimizer,
#       noise_multiplier=?,
#       max_grad_norm=?,
#       loss_reduction='sum'
#   ) 

epochs = 500
for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(X, y, gap_model, loss_fn, optimizer)
    if t % 10 == 0:
      test(X, y, gap_model, loss_fn)
print("Done!")

IndexError: ignored

## Backup

In [ ]:
# # train
# def train(dataloader, model, loss_fn, optimizer, print_every = 100):
#     size = len(dataloader.dataset)
#     model.train()
#     for batch, (X, y) in enumerate(dataloader):
#         X, y = X.to(device), y.to(device)

#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if batch % print_every == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# # test
# def test(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     test_loss, correct = 0, 0
#     with torch.inference_mode():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")